In [1]:
# Install and import TF2
!pip install -q tensorflow==2.0.0
import tensorflow as tf
print(tf.__version__)

     |████████████████████████████████| 86.3MB 70kB/s 
     |████████████████████████████████| 450kB 51.4MB/s 
     |████████████████████████████████| 3.8MB 39.1MB/s 
     |████████████████████████████████| 81kB 11.0MB/s 
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.10.0 which is incompatible.
2.0.0


In [0]:
# additional imports
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.layers import Input,Dense,Conv2D,Flatten, Dropout, GlobalMaxPooling2D, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model

In [3]:
# Load in the Data
cifar10 = tf.keras.datasets.cifar10

(x_train,y_train), (x_test,y_test) = cifar10.load_data()
x_train,x_test = x_train/255.0, x_test/255.0
y_train, y_test = y_train.flatten(), y_test.flatten()
print("x_train.shape", x_train.shape)
print("y_train.shape", y_train.shape)

170500096/170498071 [==============================] - 4s 0us/step
x_train.shape (50000, 32, 32, 3)
y_train.shape (50000,)


In [4]:
# number of classes
K = len(set(y_train))
print("number of classes", K)

number of classes 10


In [0]:
# Build the model using the functional API
def create_model():
  i = Input(shape=x_train[0].shape)
  
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(i)
  x = BatchNormalization()(x)
  x = Conv2D(32, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
  x = BatchNormalization()(x)
  x = MaxPooling2D((2, 2))(x)

  x = Flatten()(x)
  x = Dropout(0.2)(x)
  x = Dense(1024, activation='relu')(x)
  x = Dropout(0.2)(x)
  x = Dense(K, activation='softmax')(x)

  model = Model(i, x)
  return model

In [6]:
strategy = tf.distribute.MirroredStrategy()

In [7]:
print(f"Number of devices: {strategy.num_replicas_in_sync}" )

Number of devices: 1


In [0]:
with strategy.scope():
  model = create_model()

  model.compile(loss="sparse_categorical_crossentropy",
                optimizer="adam",
                metrics=["accuracy"])

In [9]:
# Fit
r = model.fit(x_train,y_train,validation_data=(x_test,y_test),batch_size=128,epochs=15)

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 542s 11ms/sample - loss: 1.3616 - accuracy: 0.5363 - val_loss: 2.8907 - val_accuracy: 0.2589
Epoch 2/15
50000/50000 [==============================] - 543s 11ms/sample - loss: 0.8356 - accuracy: 0.7054 - val_loss: 0.7903 - val_accuracy: 0.7298
Epoch 3/15
50000/50000 [==============================] - 549s 11ms/sample - loss: 0.6480 - accuracy: 0.7722 - val_loss: 0.9998 - val_accuracy: 0.6663
Epoch 4/15
50000/50000 [==============================] - 549s 11ms/sample - loss: 0.5342 - accuracy: 0.8130 - val_loss: 0.7058 - val_accuracy: 0.7573
Epoch 5/15
50000/50000 [==============================] - 548s 11ms/sample - loss: 0.4417 - accuracy: 0.8447 - val_loss: 0.6311 - val_accuracy: 0.7901
Epoch 6/15
50000/50000 [==============================] - 546s 11ms/sample - loss: 0.3690 - accuracy: 0.8691 - val_loss: 0.6016 - val_accuracy: 0.8044
Epoch 7/15
50000/50000 [====================

In [1]:
50000/391

127.8772378516624

In [2]:
10000/79

126.58227848101266

In [0]:
# Compare this to non-distributed training
model2 = create_model()
model2.compile(loss='sparse_categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'])
r = model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5)